# 【居酒屋レコメンド】
### お勧めの居酒屋をディープラーニングの自然言語処理で抽出する
### 最終的には同じカテゴリーで抽出した居酒屋と、自然減言語で抽出した居酒屋の特徴の文字を比較してみる

## ■使用データ
- グルメサイトにある、口コミの文章をスクレイピングして使用する
- モデルは文章をベクトル化して、比較出来るDoc2Vecを使用する
- スクレイピングを行うのは、食べログの兵庫県の居酒屋とする

## ■仮説
- カテゴリーで抽出したお店は主に、料理が似たお店、口コミは安さや、お店の雰囲気、接客など幅広い情報で似たお店が抽出される

## 使用外部ライブラリ

- requests==2.22.0
- beautifulsoup4==4.8.2
- gensim==3.8.3
- Janome==0.3.10

In [42]:
import requests
from bs4 import BeautifulSoup
from time import sleep

import collections
from gensim.models.doc2vec import TaggedDocument,Doc2Vec
import re
from janome.tokenizer import Tokenizer

- 食べログのサイトから、店名とお店の種類、場所を口コミのURLからスクレイピングで取得する
- 店名とお店の種類、場所の3つの文字を合体して1つの要素にして、リストに追加していく
- 口コミのURLもリストに追加していく
- 兵庫県居酒屋のpageは60あるが、お店の数が少なくなるの可能性もあるので、スクレイピングするpageの数は50件とする、

In [43]:
url = 'https://tabelog.com/hyogo/A2801/A280101/rstLst/izakaya/'
izakaya_name = []
kuchikomi_url = []
for izakaya_url in range(50):
    izakaya_url =  izakaya_url + 1 
    taberogu_page_url = requests.get(url + str(izakaya_url) + "/")
    get_code =BeautifulSoup(taberogu_page_url.text,'html.parser')
    for izakaya in get_code.find_all(class_ ='list-rst__rst-name-wrap'):
        izakaya_url = izakaya.find(class_ ='list-rst__rst-name-target cpy-rst-name')
        izakaya_one_name = izakaya.find(class_ ='list-rst__rst-name-target cpy-rst-name').text
        izakaya_kind = izakaya.find(class_='list-rst__area-genre cpy-area-genre').text
        izakaya_infomation = izakaya_one_name + "/"+izakaya_kind
        izakaya_name.append(izakaya_infomation)
        kuchikomis_url = izakaya_url.get("href")
        get_kuchikomis_url = kuchikomis_url +("dtlrvwlst/")
        kuchikomi_url.append(get_kuchikomis_url)
        sleep(0.5)

- 取得した店名とお店の種類、場所を改行をなくし、空白も除去して見やすくする

In [44]:
izakaya_name = [s.replace('\n', '') for s in izakaya_name]
izakaya_name = [s.replace(' ', '') for s in izakaya_name]
izakaya_name = [s.replace('　', '') for s in izakaya_name]


- 取得したお店情報を頭５件だけ確認する
- 取得出来た、店舗数を確認する

In [45]:
print("最初の５件のお店：")
print("")
for infomation in izakaya_name[:5]:
    print(infomation)
    print("*"*80)

print("取得した店舗数", len(izakaya_name))

最初の５件のお店：

焔立つ肉寿司と牛タンタワー肉処双葉三宮/神戸三宮（阪急）駅115m/居酒屋、しゃぶしゃぶ、焼肉
********************************************************************************
神戸焼肉みつわ屋/三宮（神戸市営）駅200m/焼肉、ホルモン、居酒屋
********************************************************************************
焼肉・にくなべ屋びいどろ北野坂店/三宮（神戸市営）駅210m/焼肉、鍋（その他）、居酒屋
********************************************************************************
とよの本舗東門店/三宮（神戸市営）駅80m/居酒屋、魚介料理・海鮮料理、鳥料理
********************************************************************************
地鶏地酒地野菜絆～ＫＩＺＵＮＡ～/神戸三宮（阪急）駅99m/居酒屋、焼鳥、日本酒バー
********************************************************************************
取得した店舗数 858


- 店、席など、どこのお店でも該当する、特徴ではなさそうな単語を予め辞書で作成する
- 次のプロセスで辞書を使い特徴から除去する
- 辞書で除外出来るのは、１文字単位なので、２文字以上の不要単語は、実際の口コミのスクレイピングの中に入力する

In [47]:
no_words =str.maketrans({'円':'','の':'','店':'','町':'','駅':'','店':'','こ':'',
                                                'さ':'','的':'','席':'','感':'','味':'','人':'','会':'','員':"","ん":"","笑":"","事":"","気":"","私":""})

- 今回取得出来た858店舗の口コミデータを取得する（ただし口コミ数が10件より少ないお店は、データ数が少ないので、除外する）
- 1つのお店の口コミのpageに、1つ1つの口コミごとにpageのURLがあるので、全てのお店の全ての口コミurlを取得する
- 1つの口コミのpageから全ての文字を取得する
- 取得した文字を形態素解析で名詞と形容詞のみを抽出する
- 単語単位で、お店を表す特徴ではない不要なデータを削除する、また正規表現で英数字なども削除する
- 最終的に店名ををキーに、口コミの単語を要素で辞書を作成する

In [48]:
t = Tokenizer()
base_url = "https://tabelog.com"
review_texts ={}


for url ,name  in list(zip(kuchikomi_url,izakaya_name))[:858]:
    one_izakaya= []
    kuchikomis_url = requests.get(url)
    kuchikomis_get_code = BeautifulSoup(kuchikomis_url.text,'html.parser')
    try:
        kuchikomis_count = int(kuchikomis_get_code.find_all(class_='c-page-count__num')[2].text)
        if kuchikomis_count > 10:
            kuchikomi_list = kuchikomis_get_code.find_all(class_='rvw-item__title-target')
            sleep(0.5)
            for reviews in (kuchikomi_list):
                one_kuchikomi_url = requests.get(base_url + reviews.get("href"))
                one_kuchikomi_code = BeautifulSoup(one_kuchikomi_url.text,'html.parser')
                get_all_words = one_kuchikomi_code.find(class_ ="rvw-item__rvw-comment").getText().replace('メニュー', "").replace('放題', "").replace('コース',"").replace('店',"").replace('料理',"").replace('町',"").replace('これ', "").replace('こちら', "").replace('良い',"").replace('こと',"").replace('注文',"").replace('さん',"").replace('時間',"")
                get_all_words = get_all_words.translate(no_words)
                edit_words = re.sub('\d', '',get_all_words)
                edit_words2 = re.sub('\W', '',edit_words)
                for token in t.tokenize(edit_words2):
                    if token.part_of_speech.split(',')[0] == '名詞':
                        one_izakaya.append(token.surface)
                    elif token.part_of_speech.split(',')[0] == '形容詞':
                        one_izakaya.append(token.surface)
                review_texts[name] = one_izakaya
                sleep(0.5)
    except:
        sleep(0.5)

- 辞書からモデルで必要なTaggedDocumentに変換

In [51]:
training_docs = []
for izakaya_name, oll_review in review_texts .items():
    add_data = TaggedDocument(words =oll_review,tags=[izakaya_name])
    training_docs.append(add_data)

- TaggedDocumentからDoc2Vecのモデルを作成
- あまり頻度数が少ない単語だと、特徴と言えないので、頻度が5以下の単語は無視する
- 次元数のvector_sizeはあまりにも次元数が多すぎると、特徴が一致しなくなる可能性があるので30次元にとどめとく

In [52]:
model = Doc2Vec(documents=training_docs, min_count=5, dm=0, vector_size=30)

- 一つのお店の次元を確認
- 30次元のnumpyデータになっているのを確認

In [53]:
model.docvecs[0]

array([ 0.52443373, -0.2777172 , -0.3984793 ,  0.22107641, -0.35084537,
       -1.102168  ,  0.28350863,  1.0433825 , -0.71851104,  0.59303856,
        0.39679778,  1.3729092 ,  0.5320931 , -0.44742706, -0.64582473,
       -0.09475201, -1.3737624 , -0.00194883,  0.6208835 ,  0.9383194 ,
       -0.42897934,  1.3373231 ,  0.34349692, -0.42666522, -0.1053613 ,
        0.36493084, -0.02021128,  1.5441809 , -0.79313564, -1.7122711 ],
      dtype=float32)

- 頭３件のお店で、次元数が近いお店ベスト５を表示
- 口コミのデータでどれだけ、カテゴリーが一致しているか確認

- 【焼肉・にくなべ屋びいどろ北野坂店】第4位 以外は焼肉店が抽出
- 【地鶏地酒地野菜絆～ＫＩＺＵＮＡ～】第2位のお店は、日本酒バーだけど、その他は海鮮がメインでカテゴリーは若干外れている
- 【北陸地魚や一豪一笑】どれも魚介料理・海鮮料理などで、一致している

- 【まとめ】口コミからもそこそこのカテゴライズが出来ることが判明

In [65]:
for rank in range(3):
    print(training_docs[rank][1],"と近いお店ベスト5")
    print(" "*120)
    result = model.docvecs.most_similar(rank)
    print("第1位",result[0])
    print("第2位",result[1])
    print("第3位",result[2])
    print("第4位",result[3])
    print("第5位",result[4])
    print("*"*120)


['焼肉・にくなべ屋びいどろ北野坂店/三宮（神戸市営）駅210m/焼肉、鍋（その他）、居酒屋'] と近いお店ベスト5
                                                                                                                        
第1位 ('炭火焼肉・にくなべ屋びいどろ本店/三宮（神戸市営）駅253m/焼肉、鍋（その他）、居酒屋', 0.9724240899085999)
第2位 ('焼肉ホルモンこうちゃん/三宮（神戸市営）駅116m/焼肉、ホルモン、居酒屋', 0.9137423038482666)
第3位 ('焼肉やまがき三宮店/神戸三宮（阪急）駅212m/焼肉、ホルモン、居酒屋', 0.8870787024497986)
第4位 ('五臓六腑三宮店/三宮（神戸市営）駅107m/居酒屋、もつ鍋、水炊き', 0.8833891749382019)
第5位 ('こだわり亭神戸三宮店/神戸三宮（阪急）駅67m/焼肉、しゃぶしゃぶ、居酒屋', 0.8797279596328735)
************************************************************************************************************************
['地鶏地酒地野菜絆～ＫＩＺＵＮＡ～/神戸三宮（阪急）駅99m/居酒屋、焼鳥、日本酒バー'] と近いお店ベスト5
                                                                                                                        
第1位 ('ウオサンジ/神戸三宮（阪急）駅208m/居酒屋、魚介料理・海鮮料理、天ぷら・揚げ物（その他）', 0.7986807823181152)
第2位 ('きさらぎ三宮店/三宮（神戸市営）駅47m/居酒屋、日本酒バー、魚介料理・海鮮料理', 0.7975310683250427)
第3位 ('しんど/三ノ宮（ＪＲ）駅487m/居酒屋、日本酒バー、魚介料理・海鮮料理', 0

- スクレイピングを行った頭3件のお店から、そのお店を最も表す単語を抽出する
- 料理名や、お店のジャンルや評価、その店を構成する単語の一覧が表示されているのを確認

In [67]:
for best_words in range(3):
    words_restaurant = training_docs[best_words][0]
    count = collections.Counter(words_restaurant)
    count = count.most_common(10)
    print(training_docs[best_words][1],"の特徴ベスト10")
    print(count)
    print("-"*140)


['焼肉・にくなべ屋びいどろ北野坂店/三宮（神戸市営）駅210m/焼肉、鍋（その他）、居酒屋'] の特徴ベスト10
[('肉', 78), ('鍋', 56), ('牛', 44), ('神戸', 44), ('個室', 28), ('焼肉', 26), ('上', 25), ('赤', 24), ('塩', 22), ('今回', 20)]
--------------------------------------------------------------------------------------------------------------------------------------------
['地鶏地酒地野菜絆～ＫＩＺＵＮＡ～/神戸三宮（阪急）駅99m/居酒屋、焼鳥、日本酒バー'] の特徴ベスト10
[('地', 51), ('鶏', 48), ('酒', 38), ('美しい', 33), ('野菜', 32), ('種', 32), ('肉', 27), ('よう', 23), ('刺身', 22), ('地酒', 21)]
--------------------------------------------------------------------------------------------------------------------------------------------
['北陸地魚や一豪一笑/三宮（神戸市営）駅89m/割烹・小料理、魚介料理・海鮮料理、居酒屋'] の特徴ベスト10
[('魚', 15), ('美しかっ', 15), ('酒', 12), ('野菜', 11), ('ソース', 11), ('塩', 11), ('お通し', 10), ('北陸', 9), ('刺身', 9), ('美しく', 9)]
--------------------------------------------------------------------------------------------------------------------------------------------


- 次元が近いお店ベスト5を表示して、そのお店の特徴を表す単語ベスト10と次元が近いお店ベスト10を確認してみる


- 【にくなべ屋びいどろ、と次元が近いお店ベストで比較してみる】
- 焼肉屋のカテゴリーだけど、鍋も扱っているお店なので、次元が近いお店１位も、焼肉で鍋の要素が大きいので納得が出来る
- 第４位は焼肉屋ではないが鍋の要素が多いのでランクインされているようだ

In [69]:
for best_words in range(1):
    words_restaurant = training_docs[best_words][0]
    count = collections.Counter(words_restaurant)
    count = count.most_common(10)
    print("■", training_docs[best_words][1],"の特徴を表す言葉")
    print(count)
    for rank in range(5):
        result = model.docvecs.most_similar(best_words)
        print(""*100)
        print("-似ている第", rank +1, "位","",result[rank],"の特徴を表す言葉")
        t = result[rank][0]
        words_restaurant =review_texts[t]
        count = collections.Counter(words_restaurant)
        count = count.most_common(10)
        print(count)
        print("*"*100)
    print("*"*100)

■ ['焼肉・にくなべ屋びいどろ北野坂店/三宮（神戸市営）駅210m/焼肉、鍋（その他）、居酒屋'] の特徴を表す言葉
[('肉', 78), ('鍋', 56), ('牛', 44), ('神戸', 44), ('個室', 28), ('焼肉', 26), ('上', 25), ('赤', 24), ('塩', 22), ('今回', 20)]

-似ている第 1 位  ('炭火焼肉・にくなべ屋びいどろ本店/三宮（神戸市営）駅253m/焼肉、鍋（その他）、居酒屋', 0.9724240899085999) の特徴を表す言葉
[('肉', 82), ('鍋', 65), ('神戸', 52), ('牛', 46), ('焼肉', 31), ('白', 23), ('上', 23), ('特', 21), ('なべ', 20), ('そ', 20)]
****************************************************************************************************

-似ている第 2 位  ('焼肉ホルモンこうちゃん/三宮（神戸市営）駅116m/焼肉、ホルモン、居酒屋', 0.9137423038482666) の特徴を表す言葉
[('ホルモン', 17), ('肉', 16), ('焼肉', 14), ('ー', 9), ('塩', 8), ('話', 8), ('神戸', 7), ('盛り', 7), ('オーナー', 7), ('そ', 7)]
****************************************************************************************************

-似ている第 3 位  ('焼肉やまがき三宮店/神戸三宮（阪急）駅212m/焼肉、ホルモン、居酒屋', 0.8870787024497986) の特徴を表す言葉
[('肉', 41), ('焼肉', 26), ('塩', 14), ('上', 14), ('牧場', 11), ('ロース', 11), ('美しい', 10), ('階', 9), ('テーブル', 9), ('美しかっ', 9)]
*******************

- 個人的に好きな和食居酒屋のわさらで口コミ特徴の次元が近いお店と、カテゴリーが近いお店を比較してみる

- 【わさらについて】
- 居酒屋であるが、ランチの定食が有名なので、特徴ベスト１位にランチが抽出されている
- そして店内で食べらる野菜は自家菜園野菜の野菜で、チキン南蛮定食が美味しいのでチキンも特徴として入っている
- カテゴリーが海鮮のお店だけど、海鮮に関する口コミはベスト10には含まれていない
- 【わさらの口コミの次元が近いお店】
- 口コミで抽出して１位に入ったお店は、同じく居酒屋で定食の特徴が入っているダイニング食卓家、同じくランチ、定食、チキン、カウンターなどの要素があって納得
- 2位3位も居酒屋だけど、ランチに力を入れているお店が揃っている
- 【わさらのカテゴリーが近いお店】
- カツオのわら焼やぶりなどが美味しいお店が抽出された、わさらと同じ海鮮料理のお店だが、わさらと比べてより海鮮要素が大きいお店だ
- 特徴は高知のカツオ関係の単語が独占している

- 【まとめ】
- カテゴリーは海鮮だが、口コミで比較すると違う視点からの類似しているお店を発見することが出来る

In [57]:
wasara = input("好きなお店") 

好きなお店わさら


In [64]:
number = 0
for k in review_texts.keys():
    if str(wasara)  in k:
        print(k)
        words_restaurant = training_docs[number][0]
        count = collections.Counter(words_restaurant)
        count = count.most_common(10)
        print(count)
        print("自然言語で抽出したお店")
        for rank in range(3):
            result = model.docvecs.most_similar(number)
            print("似ている第", rank +1, "位","、",result[rank],"の特徴を表す言葉")
            t = result[rank][0]
            words_restaurant2 =review_texts[t]
            count2 = collections.Counter(words_restaurant2)
            count2 = count2.most_common(10)
            print(count2)
            words_restaurant 
            
            print("*"*100)
        print("カテゴリーで近いお店")
        for n in review_texts.keys(): 
            if n == k:
                print("")
            elif k.split("/")[2]  in n:
                print("*"*100)
                print(n)
                f= review_texts[n]
                count = collections.Counter(f)
                count = count.most_common(10)
                print(count)
                print("*"*100)
        numer = number +1
    else :
        number = number +1

わさら/神戸三宮（阪神）駅209m/居酒屋、魚介料理・海鮮料理、郷土料理（その他）
[('ランチ', 20), ('神戸', 19), ('定食', 14), ('野菜', 13), ('カウンター', 12), ('ご飯', 11), ('日替わり', 11), ('チキン', 10), ('普通', 10), ('鶏', 10)]
自然言語で抽出したお店
似ている第 1 位 、 ('ダイニング食卓家/三宮・花時計前駅144m/居酒屋', 0.9487371444702148) の特徴を表す言葉
[('定食', 12), ('チキン', 11), ('ご飯', 11), ('ランチ', 9), ('時', 8), ('南蛮', 8), ('喫煙', 6), ('ボリューム', 6), ('居酒屋', 6), ('カウンター', 5)]
****************************************************************************************************
似ている第 2 位 、 ('ごはんや一芯神戸/三ノ宮（ＪＲ）駅75m/居酒屋、創作料理、日本酒バー', 0.9404372572898865) の特徴を表す言葉
[('ランチ', 37), ('ご飯', 35), ('美しい', 26), ('一', 23), ('焼き', 18), ('日替わり', 17), ('定食', 17), ('日', 17), ('小鉢', 16), ('時', 15)]
****************************************************************************************************
似ている第 3 位 、 ('いちにっさん/三ノ宮（ＪＲ）駅108m/和食（その他）、居酒屋、創作料理', 0.9327062964439392) の特徴を表す言葉
[('ランチ', 34), ('定食', 21), ('壱', 16), ('カウンター', 13), ('一', 12), ('野菜', 12), ('鶏', 11), ('いい', 11), ('焼き', 11), ('日', 11)]
******************

-  チェーン店の鳥貴族でも口コミ特徴の次元が近いお店と、カテゴリーが近いお店を比較してみる

- 【鳥貴族について】
- コスパ最高のお店なので、やはり安いの特徴を確認出来る
- 焼き鳥関係の単語が多く占めていて、特徴としては専ら焼き鳥の単語が多い
- 鳥貴族は色んなアルコールがあるがビールが特徴に入るのを確認
- 【鳥貴族の口コミの次元が近いお店】
- 2位は同じ鳥貴族の別支店がランクイン
- 3位の炭火焼鳥ちんどん三宮店は安い、焼き鳥、ビールの単語が入っているので、納得

- 【鳥貴族のカテゴリーが近いお店】
- 炭玄は焼き鳥屋だが、刺身の特徴があったり酒の特徴があったり若干違う特徴があるようだ
- カテゴリーが同じなので、もちろん別支店の鳥貴族が抽出されている
- にはとりや三宮2号店は同じ焼き鳥要素は多いが、安い特徴はなかった、たまたまなのかもしれない

- 【まとめ】
- チェーン店の同じ系統のお店は、口コミも次元が近いのを確認
- 居酒屋は色んな種類のお酒があるが、特定のお酒を売りにしていないお店でも口コミからそのお店の特徴のお酒が抽出することも出来る

In [124]:
torikizoku = input("好きなお店") 

好きなお店鳥貴族JR三ノ宮店


In [125]:
print("自然言語で抽出したお店")
number = 0
for k in review_texts.keys():
    if str(torikizoku)  in k:
        print(k)
        print(number) 
        words_restaurant = training_docs[number][0]
        count = collections.Counter(words_restaurant)
        count = count.most_common(10)
        print(count)
        for rank in range(3):
            result = model.docvecs.most_similar(number)
            print(""*100)
            print("似ている第", rank +1, "位","、",result[rank],"の特徴を表す言葉")
            t = result[rank][0]
            words_restaurant2 =review_texts[t]
            count2 = collections.Counter(words_restaurant2)
            count2 = count2.most_common(10)
            print(count2)
            words_restaurant 
            
            print("*"*100)
        print("カテゴリーで近いお店")
        for n in review_texts.keys(): 
            if n == k:
                print("")
            elif k.split("/")[2]  in n:
                print("*"*100)
                print(n)
                f= review_texts[n]
                count = collections.Counter(f)
                count = count.most_common(10)
                print(count)
                print("*"*100)
        numer = number +1
    else :
        number = number +1

自然言語で抽出したお店
鳥貴族JR三ノ宮店/三宮（神戸新交通）駅189m/焼鳥、鳥料理、居酒屋
176
[('鳥', 14), ('貴族', 11), ('安い', 7), ('前', 6), ('年', 6), ('美しい', 6), ('焼鳥', 5), ('ビール', 5), ('焼き鳥', 5), ('いい', 5)]

似ている第 1 位 、 ('鳥貴族三宮3号店/三宮（神戸市営）駅39m/焼鳥、鳥料理、居酒屋', 0.9845262765884399) の特徴を表す言葉
[('鳥', 23), ('貴族', 21), ('時', 10), ('安い', 10), ('一', 10), ('いい', 8), ('飲', 7), ('三宮', 6), ('深夜', 6), ('後', 6)]
****************************************************************************************************

似ている第 2 位 、 ('鳥貴族阪急三宮駅前店/神戸三宮（阪急）駅115m/焼鳥、鳥料理、居酒屋', 0.9833000898361206) の特徴を表す言葉
[('貴族', 17), ('鳥', 15), ('安い', 10), ('焼き鳥', 8), ('串', 7), ('o', 7), ('ない', 6), ('なか', 5), ('焼鳥', 5), ('時', 5)]
****************************************************************************************************

似ている第 3 位 、 ('炭火焼鳥ちんどん三宮店/三宮（神戸新交通）駅300m/居酒屋、焼鳥、鍋（その他）', 0.9721778631210327) の特徴を表す言葉
[('焼き鳥', 12), ('よう', 11), ('安い', 7), ('何', 7), ('一', 6), ('なく', 6), ('個', 6), ('ビール', 6), ('今回', 6), ('三宮', 5)]
***************************************************


## 【総合まとめ】
- 口コミからも、カテゴリーをある程度抽出することが出来るが、必ずしも一致していない
- カテゴリーで似ているお店を見つけるよりも、口コミのデータを利用した方が、近いお店を見つけられる可能性もある
- カテゴリーも、口コミも、組み合わせることにより、よりお勧めのお店を見つけることが出来る
- 飲食店だけではなく、映画や、商品、サービスの口コミなどから、似ているものを見つけることが出来る可能性がある